In [44]:
import os
import pandas as pd
import warnings
import functions as fc

In [45]:
#Ocultar advertencias/warnings mas no los borra
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

In [46]:
#Año evaluado
Añoeval = 2025

**Variables globales de las rutas necesarias**

In [47]:
#Rutas del archivo de la COOISPI
Ruta_COOISPI = "G:/.shortcut-targets-by-id/1rqpfbdZ6z51epFv6ZwognhckW7HqBMjN/SIRI_2024_INFORMES_SOBREPESO/DATOS/Datos_COOISPI_Empaques2.csv"                # Ruta COOISPI en csv
#Ruta_COOISPI_2024="/content/drive/MyDrive/Consolidado COOISPI/2024/Z10.xlsx"                                                                                # Ruta COOISPI en el 2024
#Ruta_COOISPI_2025="/content/drive/MyDrive/Consolidado COOISPI/2025/Z10.xlsx"                                                                                # Ruta COOISPI en el 2025

#Rutas del archivo del EGE
Ruta_Archivo_EGE = "G:/.shortcut-targets-by-id/1rqpfbdZ6z51epFv6ZwognhckW7HqBMjN/SIRI_2024_INFORMES_SOBREPESO/DATOS/Datos_Emp2_2021.csv"
#Ruta_Archivo_EGE="/content/drive/MyDrive/SIRI_2024_INFORMES_SOBREPESO/DATOS/Datos_Emp2_2021.csv"                                                            # En formato csv en el drive de Melanny Herrera
#Ruta_Archivo_EGE="/content/drive/MyDrive/appsheet/Coolab/Documentos/Empaques2/DB Empaque de Estrella 2021_Copia.xlsm"                                       # Ruta EGE pruebas Melanny Herrera
#Ruta_Archivo_EGE="/content/drive/MyDrive/EGE Colcafé Medellín 2018 - 2024/Data historica/2021-2024/DB Empaque de Estrella 2021.xlsm"                        # Ruta EGE en el drive de Jesse Mauricio Beltran

#Rutas del archivo de Novedades
Ruta_Novedades = "G:/.shortcut-targets-by-id/1rqpfbdZ6z51epFv6ZwognhckW7HqBMjN/SIRI_2024_INFORMES_SOBREPESO/INFORME_EMPAQUES_2/Empaques_2_Novedades.xlsx"
#Ruta_Novedades = "/content/drive/MyDrive/SIRI_2024_INFORMES_SOBREPESO/INFORME_EMPAQUES_2/Empaques_2_Novedades.xlsx"                                         # Ruta del archivo de Novedades en el drive de Melanny Herrera
#Ruta_Novedades = '/content/drive/MyDrive/Proyectos/2024/SIRI_2024_INFORMES_SOBREPESO/INFORME_EMPAQUES_2/Empaques_2_Novedades.xlsx'                          # Ruta del archivo de Novedades en el drive de Jesse Mauricio Beltran

#Rutas del archivo de Consolidados
Ruta_Consolidados = "G:/.shortcut-targets-by-id/1rqpfbdZ6z51epFv6ZwognhckW7HqBMjN/SIRI_2024_INFORMES_SOBREPESO/INFORME_EMPAQUES_2/Consolidado_V2.xlsx"
#Ruta_Consolidados ="/content/drive/MyDrive/SIRI_2024_INFORMES_SOBREPESO/INFORME_EMPAQUES_2/Consolidado_V2.xlsx"                                             # Ruta del archivo de Consolidados en el drive de Melanny Herrera
#Ruta_Consolidados ='/content/drive/MyDrive/Proyectos/2024/SIRI_2024_INFORMES_SOBREPESO/INFORME_EMPAQUES_2/Consolidado.xlsx'                                 # Ruta del archivo de Novedades en el drive de Jesse Mauricio Beltran

#Ruta de salida
Ruta_Emp_2 = "G:/.shortcut-targets-by-id/1rqpfbdZ6z51epFv6ZwognhckW7HqBMjN/SIRI_2024_INFORMES_SOBREPESO/INFORME_EMPAQUES_2/"
#Ruta_Emp_2 = "/content/drive/MyDrive/SIRI_2024_INFORMES_SOBREPESO/INFORME_EMPAQUES_2/"                                                                      # Ruta del archivo consolidado csv en el drive de Melanny Herrera
#Ruta_Emp_2 = "/content/drive/MyDrive/SIRI_2024_INFORMES_SOBREPESO/INFORME_EMPAQUES_2/"                                                                      # Ruta del archivo consolidado csv en el drive de Jesse Mauricio Beltran

#Validacion de existencia de las rutas
for nombre, ruta in {
  "Ruta_COOISPI": Ruta_COOISPI,
  "Ruta_Archivo_EGE": Ruta_Archivo_EGE,
  "Ruta_Novedades": Ruta_Novedades,
  "Ruta_Consolidados": Ruta_Consolidados,
  "Ruta_Emp_2": Ruta_Emp_2,
}.items():
  if not os.path.exists(ruta):
    raise FileNotFoundError(f"La ruta '{nombre}' no fue encontrada.")
  else:
    print(f"Ruta '{nombre}' encontrada.")

Ruta 'Ruta_COOISPI' encontrada.
Ruta 'Ruta_Archivo_EGE' encontrada.
Ruta 'Ruta_Novedades' encontrada.
Ruta 'Ruta_Consolidados' encontrada.
Ruta 'Ruta_Emp_2' encontrada.


In [48]:
#Leer_archivo COOISPI csv 
df_COOISPI_COMB = fc.leer_archivo(Ruta_COOISPI)

Leyendo archivo...


In [49]:
#Convertir columna 'Codigo' a str para futuros procesor (merge y tabla pivote)
df_COOISPI_COMB['Codigo'] = df_COOISPI_COMB['Codigo'].astype(str)

In [50]:
#Funcion para leer archivo el EGE/TPM
df_Emp_2 = fc.leer_archivo(Ruta_Archivo_EGE, 'Datos')

Leyendo archivo...


In [51]:
#Reemplazar el nombre de la columna "Día:" para estandarizar los scripts
df_Emp_2.rename(columns={"Día:":"Dia"}, inplace=True)

In [52]:
#Eliminar_filas_vacias
numeros_vacios_filtrados = fc.eliminar_filas_vacias(df_Emp_2, 'Número')

In [53]:
# #Filtrando el DataFrame según las condiciones dadas
# df_filtrado = df_Emp_2.loc[
#     (df_Emp_2['Año:'] == 2024) &
#     (df_Emp_2['Mes:'] == "Abril") &
#     (df_Emp_2['Código y Descrip. / Producto:'] == "1040596 Cafe COLCAFE Clasic Grnel 2500X1.5g esp")
# ]

# #Filtrando el DataFrame según las condiciones dadas
# df_filtrado = df_Emp_2.loc[
#     (df_Emp_2['Año:'] == 2024) &
#     (df_Emp_2['Mes:'] == "Enero") &
#     (df_Emp_2['Código y Descrip. / Producto:'] == "1056678 Cafe COLCAFE clasico 1.5g 48sob 30ple")
# ]
#
# # Calculando la suma de la columna específica
# suma_unidades_producidas = df_filtrado['Unidades Producidas (Conformes) :'].sum()
#
# # Mostrando la suma
# print(suma_unidades_producidas)

In [54]:
#Crear_fecha
df_Emp_2 = fc.crear_fecha(df_Emp_2)
df_Emp_2.head()

,Número,Dia,Mes:,Año:,Semana:,Turno:,Hora de Inicio:,Hora Final:,Máquina / Equipo:,Código y Descrip. / Producto:,...,OBSERVACIONES:,Legal Contractual\nPeso min,Legal Contractual\nMarcación min,Legal Contractual\nOtros min,Legal Contractual\nPeso Ev,Legal Contractual\nMarcación Ev,Legal Contractual\nOtros Ev,Kg Producidos (Conforme),IdMes,Fecha
0,18036,4,Enero,2021,1.0,3.0,21:10:07,05:12:51,TOYO 3,1038938 Cafe COLCAFE clasico 1.5g 48sob 30ple,...,.E.D.@_@,NaN,NaN,NaN,NaN,NaN,NaN,254.40000,1,2021-01-04
1,18037,4,Enero,2021,1.0,2.0,13:15:00,21:10:07,TOYO 3,1038938 Cafe COLCAFE clasico 1.5g 48sob 30ple,...,..@_@.E.D.@_@,NaN,NaN,NaN,NaN,NaN,NaN,251.90808,1,2021-01-04
2,18038,4,Enero,2021,1.0,2.0,13:20:34,21:10:05,TOYO 4,1038938 Cafe COLCAFE clasico 1.5g 48sob 30ple,...,.BOBINA@_@.COMIDA.@_@.DESENHEBRE DE PISTAS 4 Y...,NaN,NaN,NaN,NaN,NaN,NaN,209.44000,1,2021-01-04
3,18039,4,Enero,2021,1.0,1.0,06:33:11,13:09:34,TOYO 4,1038938 Cafe COLCAFE clasico 1.5g 48sob 30ple,...,.mordazas sucias por finos del cafe@_@.PISTAS ...,NaN,NaN,NaN,NaN,NaN,NaN,207.76800,1,2021-01-04
4,18040,4,Enero,2021,1.0,1.0,06:00:30,13:10:21,TOYO 3,1038938 Cafe COLCAFE clasico 1.5g 48sob 30ple,...,.(Tec. Mto: Técnico Mantenimiento) Se realiza ...,NaN,NaN,NaN,NaN,NaN,NaN,180.55200,1,2021-01-04


In [55]:
#Tomar solo las columnas de interes del informe de Empaques2 para el sobrepeso
Columnas=['Número','Dia','Fecha','Mes:','IdMes','Año:','Máquina / Equipo:','Semana:','Turno:','Código y Descrip. / Producto:','Unidades Producidas (Conformes) :','Peso Promedio de la unidad (K):','Gramaje (K):']
df_Emp_2 = df_Emp_2[Columnas]

In [56]:
#Filtrar_desde_añio
df_Emp_2 = fc.filtrar_desde_anio(df_Emp_2, 2024)

In [57]:
#Especificar las columnas en las que deseas aplicar el filtro para eliminar los valores nulos de las columnas de Gramaje, Unidades producidas y Peso promedio
columnas_filtrar_nulos = ['Gramaje (K):', 'Unidades Producidas (Conformes) :', 'Peso Promedio de la unidad (K):']

In [58]:
#Filtrar_nulos
df_Emp_2_nan_ceros, df_Emp_2 = fc.filtrar_nulos(df_Emp_2, columnas_filtrar_nulos)

**Modificación de los tipos de datos del archivo de TPM de EMPAQUES 2**

In [59]:
#Validar_numericos
df_non_conver_a, df_Emp_2 = fc.validar_numericos(df_Emp_2, 'Peso Promedio de la unidad (K):')

In [60]:
#Validar_numericos
df_non_conver_b, df_Emp_2 = fc.validar_numericos(df_Emp_2, 'Gramaje (K):')

In [61]:
#Extraer_codigo
df_Emp_2 = fc.extraer_codigo(df_Emp_2)

**Identificación y supresión de aquellos códigos con valores nulos**

In [62]:
#Eliminar_codigos_nan
df_Emp_2, df_Emp_2_Codigos_nan = fc.eliminar_codigos_nan(df_Emp_2)

In [63]:
filtro_nan_ceros = df_Emp_2[columnas_filtrar_nulos].isna().any(axis=1) | (df_Emp_2[columnas_filtrar_nulos] == 0).any(axis=1)
df_Emp_2_nan_ceros = df_Emp_2[filtro_nan_ceros]
df_Emp_2 = df_Emp_2[~filtro_nan_ceros]

In [64]:
#Calcular_columnas
df_Emp_2 = fc.calcular_columnas(df_Emp_2)

**Generación de Novedades**

In [65]:
Sobrepeso_Novedades = 0.1  #10% de Sobrepeso

In [ ]:
#Generar_novedades
df_Emp_2_Nov_Sobrepeso = fc.generar_novedades(df_Emp_2, Sobrepeso_Novedades)
# df_Emp_2_Nov_Sobrepeso.head()

,Número,Dia,Fecha,Mes:,IdMes,Año:,Máquina / Equipo:,Semana:,Turno:,Código y Descrip. / Producto:,Unidades Producidas (Conformes) :,Peso Promedio de la unidad (K):,Gramaje (K):,Codigo,Real_empaque_calculado,Debe_ser_empaque_calculado,Diferencia_calculado,Sobrepeso_calculado
10938,29051,27,2025-03-27,Marzo,3,2025,TOYO 4,13.0,2.0,1038372 Cafe COLCAFE Granul 24x32x1.5g Int Ing,6968.0,0.00168,0.0015,1038372,11.70624,10.452,1.25424,0.120000
11045,29158,12,2025-04-12,Abril,4,2025,TOYO 4,15.0,3.0,1056679 Cafe COLCAFE Granulado 1.5g 48sob 30ple,136912.0,0.00167,0.0015,1056679,228.64304,205.368,23.27504,0.113333


In [67]:
#Agregar columna 'Costo/kg'
df_Emp_2['Costo/kg'] = 0.0

**Tabla de Consolidado Mensual**

In [68]:
#Pivote_consolidado
df_Emp_2_Mes = fc.pivote_consolidado(df_Emp_2)

In [69]:
#Tomar datos desde el año 2024 en adelante
df_Emp_2_Mes = fc.filtrar_desde_anio(df_Emp_2_Mes, 2024)

**Creación de la agrupación Mensual del archivo de semielaborados**

In [70]:
df_costo_semi = df_COOISPI_COMB.copy(deep=True)

In [71]:
#Creacion de la agrupacion mensual del archivo semielaborado
df_costo_semi_Mes = fc.pivote_semielaborados(df_costo_semi)

**Generación del nuevo archivo consolidado V2**

In [72]:
df_Emp2_2 = df_Emp_2.copy(deep=True)

In [73]:
#Traer la columna 'Costo/kg' al dataframe original
df_Emp2_2 = fc.merge_costo(df_Emp2_2, df_costo_semi_Mes)

In [74]:
#Eliminar la columna Costo/kg antigua y renombrar la nueva
df_Emp2_2 = fc.modificar_costo(df_Emp2_2)

In [75]:
#Definir meta sobrepeso
df_Emp2_2['Meta'] = 0.015

In [ ]:
#Calcular columna 'Ahorros/Perdidas'
df_Emp2_2 = fc.calcular_ahorros_perdidas(df_Emp2_2)
# df_Emp2_2.head()

,Número,Dia,Fecha,Mes:,IdMes,Año:,Máquina / Equipo:,Semana:,Turno:,Código y Descrip. / Producto:,...,Peso Promedio de la unidad (K):,Gramaje (K):,Codigo,Real_empaque_calculado,Debe_ser_empaque_calculado,Diferencia_calculado,Sobrepeso_calculado,Costo/kg,Meta,Ahorros_Perdidas
0,25536,2,2024-01-02,Enero,1,2024,TOYO 3,1.0,1.0,1056678 Cafe COLCAFE clasico 1.5g 48sob 30ple,...,0.00151,0.0015,1056678,63.56496,63.1440,0.42096,0.006667,40412.642311,0.015,21406.899933
1,25537,2,2024-01-02,Enero,1,2024,TOYO 3,1.0,2.0,1056678 Cafe COLCAFE clasico 1.5g 48sob 30ple,...,0.00151,0.0015,1056678,264.41459,262.6635,1.75109,0.006667,40412.642311,0.015,89047.435396
2,25538,2,2024-01-02,Enero,1,2024,TOYO 3,1.0,3.0,1056678 Cafe COLCAFE clasico 1.5g 48sob 30ple,...,0.00150,0.0015,1056678,277.20000,277.2000,0.00000,0.000000,40412.642311,0.015,168035.766730
3,25539,2,2024-01-02,Enero,1,2024,TOYO 4,1.0,1.0,1057623 Café Sprouts Dark Organic 12x2gx24UN,...,0.00200,0.0020,1057623,0.00200,0.0020,0.00000,0.000000,97977.000000,0.015,2.939310
4,25540,2,2024-01-02,Enero,1,2024,TOYO 4,1.0,2.0,1057623 Café Sprouts Dark Organic 12x2gx24UN,...,0.00200,0.0020,1057623,156.20800,156.2080,0.00000,0.000000,97977.000000,0.015,229571.868240


In [ ]:
# df_Emp2_2.dtypes

Número                                 int64
Dia                                    int64
Fecha                                 object
Mes:                                  object
IdMes                                  int64
Año:                                   int64
Máquina / Equipo:                     object
Semana:                              float64
Turno:                               float64
Código y Descrip. / Producto:         object
Unidades Producidas (Conformes) :    float64
Peso Promedio de la unidad (K):      float64
Gramaje (K):                         float64
Codigo                                object
Real_empaque_calculado               float64
Debe_ser_empaque_calculado           float64
Diferencia_calculado                 float64
Sobrepeso_calculado                  float64
Costo/kg                             float64
Meta                                 float64
Ahorros_Perdidas                     float64
dtype: object

**Unión de los DataFrames en un DataFrame totalizado**

In [79]:
#Crear copia de df_Emp_2_Mes
df_totalizado_Mes = df_Emp_2_Mes.copy(deep=True)

In [80]:
#Validar que no hay filas repetidas en df_totalizado_Mes
fc.filas_repetidas(df_totalizado_Mes)

No hay filas duplicadas.


In [81]:
#Fusionar los DataFrames en base a 'Año:', 'Codigo' y 'Mes:'
df_totalizado_Mes = fc.unir_dataframes(df_totalizado_Mes, df_costo_semi_Mes)

In [ ]:
#Metodo calcular_columnas_totalizado con meta fija
df_totalizado_Mes = fc.calcular_columnas_totalizado(df_totalizado_Mes, 0.015)
# df_totalizado_Mes.head()

,Año:,Mes:,Codigo,Máquina / Equipo:,Debe_ser_empaque_calculado,Diferencia_calculado,Gramaje (K):,Peso Promedio de la unidad (K):,Real_empaque_calculado,Unidades Producidas (Conformes) :,Ctd.,Impte.ML,Real_empaque,Debe_ser_empaque,Diferencia,Sobrepeso,Meta,Costo/kg,Ahorros_Perdidas
0,2024,Abril,1038372,TOYO 4,58.212,0.38808,0.0015,0.001510,58.60008,38808.0,68.0,2925700.0,58.60008,58.212,0.38808,0.006667,0.015,43025.0,21010.570350
1,2024,Abril,1038375,TOYO 4,231.324,2.63112,0.0015,0.001515,233.95512,154216.0,250.0,10310750.0,233.95512,231.324,2.63112,0.011374,0.015,41243.0,34985.611082
2,2024,Abril,1040596,TOYO 4,19.314,0.38628,0.0015,0.001530,19.70028,12876.0,20.0,772880.0,19.70028,19.314,0.38628,0.020000,0.015,38644.0,-3806.488102
3,2024,Abril,1051754,TOYO 4,30.084,0.80224,0.0015,0.001540,30.88624,20056.0,35.0,1505875.0,30.88624,30.084,0.80224,0.026667,0.015,43025.0,-15503.605553
4,2024,Abril,1056664,TOYO 4,30.000,7.00000,0.0015,0.001850,37.00000,20000.0,56.0,4703328.0,37.00000,30.000,7.00000,0.233333,0.015,83988.0,-678483.060000


In [83]:
#Obtener Codigos unicos | Cod. Descrip. | Maquina con un merge
df_Codigos_Emp_2 = fc.obtener_cod_descrip(df_costo_semi, df_Emp_2_Mes)

In [84]:
#Obtener Maquinas únicas de TPM o df_codigos_desc_maq_Mezclas
df_Maquinas_Emp_2 = fc.eliminar_duplicados_columna(df_Codigos_Emp_2, 'Máquina / Equipo:')

In [85]:
#Unión de los dataframes de no convertibles (validar_y_convertir_datos)
df_non_conver = pd.concat([df_non_conver_a, df_non_conver_b], ignore_index=True)

**Generación de un archivo de fechas Automáticas**

In [86]:
#Generar dataframe con fechas dese 2024-01-01 hasta el dia actual
df_Fechas = fc.generar_fechas("2024-01-01")

In [87]:
# print(df_Emp_2.shape)
# print(df_COOISPI_COMB.shape)
# print(df_Emp_2_Mes.shape)
# print(df_Codigos_Emp_2.shape)
# print(df_Maquinas_Emp_2.shape)
# print(df_totalizado_Mes.shape)
# print(df_Fechas.shape)
# print(df_Emp2_2.shape)
# print()
# print(df_Emp_2_Nov_Sobrepeso.shape)
# print(df_Emp_2_Codigos_nan.shape)
# #print(df_Mol_Codigos_unicos.shape)
# print(df_Emp_2_nan_ceros.shape)
# print(df_non_conver.shape)

**Consolidado csv**

In [88]:
#Ruta para consolidado df_Emp2_2
df_Emp2_2.to_csv(Ruta_Emp_2 + "DB_Emp2_Out_vf.csv", index=False)

**Archivo de consolidado**

In [89]:
#Diccionario de hojas para el archivo de consolidado
dataframes_consolidado = {
    "Hoja1": df_Emp_2,
    "Hoja2": df_COOISPI_COMB,
    "Hoja3": df_Emp_2_Mes,
    "Hoja4": df_Codigos_Emp_2,  #df_merged
    "Hoja5": df_Maquinas_Emp_2,
    "Hoja6": df_totalizado_Mes,
    "Hoja7": df_Fechas,
    "Hoja8": df_Emp2_2,
    "Hoja9": df_Emp2_2
}
#Actualizar hojas del excel de Consolidado_V2
fc.actualizar_hojas_excel(Ruta_Consolidados, dataframes_consolidado)

Procesando Consolidado...
✅ Las hojas en el Archivo de Consolidado han sido limpiadas y actualizadas correctamente.


**Archivo de novedades : Este archivo compila las diferentes novedades presentes en los archivos de TPM**

In [90]:
#Diccionario de hojas para el archivo de novedades
dataframes_novedades = {
    "Hoja1": df_Emp_2_Nov_Sobrepeso,          #Novedades sobrepeso
    "Hoja2": df_Emp_2_Codigos_nan,            #Novedades codigos nulos
    #"Hoja3": df_Emp_2_Codigos_unicos,        #Codigos presentes en TPM y no en COOISPI
    "Hoja4": df_Emp_2_nan_ceros,              #Valores nulos o ceros para gramaje, unidades producidas y promedio peso unidad
    "Hoja5": df_non_conver                    #Valores no convertibles
}
#Guardar el archivo de Novedades
fc.crear_archivo_novedades(Ruta_Novedades, dataframes_novedades)

Procesando Novedades...
✅ El Archivo de Novedades guardado exitosamente con 4 hojas guardadas correctamente.
